In [1]:
import altair as alt
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import panel as pn
from IPython.display import display, HTML

In [2]:
# %load_ext watermark
# %watermark -v -m -p altair,ipywidgets,matplotlib,numpy,pandas,panel,IPython,vega
# %watermark -u -n -t -z

In [3]:
# Suppress warnings.
import warnings
warnings.filterwarnings("ignore")

# Suppress js errors.
alt.renderers.enable('jupyterlab')

# Wrap ipywidgets with panel
pn.extension('ipywidgets')

In [4]:
DATA_DIR = 'csv/'

### Data: Olou

In [5]:
olou = pd.read_csv(DATA_DIR + 'Olou_counts.csv', parse_dates=['Date'])
olou.set_index('Date', drop=False, inplace=True)

In [6]:
olou['dataset'] = 'olou'
olou['date_str'] = olou['Date'].astype(str)
olou['year'] = olou['date_str'].str[:4]
olou['month'] = pd.to_numeric(olou['date_str'].str[5:7])

# This could be more elegant.
olou.loc[olou['month'] <= 12, 'quarter'] = 4
olou.loc[olou['month'] <= 9, 'quarter'] = 3
olou.loc[olou['month'] <= 6, 'quarter'] = 2
olou.loc[olou['month'] <= 3, 'quarter'] = 1

In [7]:
# olou.head()

### Data: Monsoon

In [8]:
monsoon = pd.read_csv(DATA_DIR + 'Monsoon_data.csv', parse_dates=['Date'])
monsoon.set_index('Date', drop=False, inplace=True)

In [9]:
monsoon['dataset'] = 'monsoon'
monsoon['date_str'] = monsoon['Date'].astype(str)
monsoon['year'] = monsoon['date_str'].str[:4]
monsoon['month'] = pd.to_numeric(monsoon['date_str'].str[5:7])

# This could be more elegant.
monsoon.loc[monsoon['month'] <= 12, 'quarter'] = 4
monsoon.loc[monsoon['month'] <= 9, 'quarter'] = 3
monsoon.loc[monsoon['month'] <= 6, 'quarter'] = 2
monsoon.loc[monsoon['month'] <= 3, 'quarter'] = 1

In [10]:
# monsoon.head(20)

### Data: Monsoon + Olou

In [11]:
dataframes = [monsoon, olou]
dataset = pd.concat(dataframes)

In [12]:
# dataset.head()

#### Define a common date range

In [13]:
# min_x is the LARGER min date of the two sets.
if olou.index.min() < monsoon.index.min():
    min_x = monsoon.index.min()
else:
    min_x = olou.index.min()

# max_x is the SMALLER max date of the two sets.
if olou.index.max() < monsoon.index.max():
    max_x = olou.index.max()
else:
    max_x = monsoon.index.max()

#### Exclude dates not in both sets

In [14]:
dataset = dataset[(dataset['Date'] >= min_x) & (dataset['Date'] <= max_x)]

In [15]:
# dataset.head()

### Selectors

In [16]:
# Selections.
select = alt.selection_interval(
    bind="scales",
    encodings=['x', 'y']
)

# Opacities
opacity = alt.binding_range(min=0, max=1, step=0.01, name='Visbility: Cumulative ⟷ Average')
opacity_val = alt.param(value=0.15, bind=opacity)

### Scales & Ranges

In [17]:
# X-axis ranges.
x_domain = [min_x, max_x]

# Y-axis ranges.
olou_domain = [dataset['Counts'].min() - 500, dataset['Counts'].max() + 500]
monsoon_domain = [dataset['Precip'].min() - 10, dataset['Precip'].max() + 10]

### Main Layers

In [18]:
# Monsoon precipitation.
line_chart_monsoon = alt.Chart(dataset[np.isnan(dataset['Precip']) == False], title='Monsoon Data').mark_line(
    interpolate='basis',
    strokeWidth=2.5,
    opacity=1 - opacity_val
).encode(
    x=alt.X('Date:T', title="", scale=alt.Scale(domain=x_domain)),
    y=alt.Y('Precip:Q', title="Precipitation (mm per month)", scale=alt.Scale(domain=monsoon_domain)),
    color=alt.Color("dataset", legend=None)
).add_params(
    opacity_val,
    select
)

# Monsoon precipitation rolling average.
line_chart_monsoon_rolling = alt.Chart(dataset[np.isnan(dataset['Precip']) == False]).transform_window(
    rolling_mean='mean(Precip)',
    frame=[-15, 15],
).mark_line(
    color='blue',
    strokeWidth=2,
    interpolate='basis',
    opacity=opacity_val
).encode(
    x=alt.X('Date:T', scale=alt.Scale(domain=x_domain)),
    y=alt.Y('rolling_mean:Q', scale=alt.Scale(domain=monsoon_domain))
).add_params(
    opacity_val,
    select
)

# Olou cosmic ray counts.
line_chart_olou = alt.Chart(dataset[np.isnan(dataset['Counts']) == False], title='Olou Data').mark_line(
    interpolate='basis',
    strokeWidth=2.5,
    opacity=1 - opacity_val
).encode(
    x=alt.X('Date:T', title="", scale=alt.Scale(domain=x_domain)),
    y=alt.Y('Counts:Q', title="Olou Count (nm per minute)", scale=alt.Scale(domain=olou_domain)),
    color=alt.Color("dataset", legend=None)
).add_params(
    opacity_val,
    select
)

# Olou cosmic ray counts rolling average.
line_chart_olou_rolling = alt.Chart(dataset[np.isnan(dataset['Counts']) == False]).transform_window(
    rolling_mean='mean(Counts)',
    frame=[-15, 15],
).mark_line(
    color='red',
    strokeWidth=2,
    interpolate='basis',
    opacity=opacity_val
).encode(
    x=alt.X('Date:T', scale=alt.Scale(domain=x_domain)),
    y=alt.Y('rolling_mean:Q', title="Olou Count (nm per minute)", scale=alt.Scale(domain=olou_domain))
).add_params(
    opacity_val,
    select
)

### Interactive Layers

In [19]:
# Get nearest point y-value from selection.
nearest = alt.selection_point(
    nearest=True,
    on='mouseover',
    fields=['Date'],
    empty=False
)

# Gray draggable bar.
rules = alt.Chart(dataset).mark_rule(
    color='lightgray',
    opacity=0.4
).encode(
    x=alt.X('Date:T', scale=alt.Scale(domain=x_domain)),
    strokeWidth=alt.StrokeWidthValue(4)
).transform_filter(
    nearest
)

# Olou selector.
selectors_olou = alt.Chart(dataset[dataset['dataset'] == 'olou']).mark_point().encode(
    x=alt.X('Date:T', scale=alt.Scale(domain=x_domain)),
    y=alt.Y('Counts:Q', scale=alt.Scale(domain=olou_domain)),
    opacity=alt.value(0),
).add_params(
    nearest
)

# Monsoon selector.
selectors_monsoon = alt.Chart(dataset[dataset['dataset'] == 'monsoon']).mark_point().encode(
    x=alt.X('Date:T', scale=alt.Scale(domain=x_domain)),
    y=alt.Y('Precip:Q', scale=alt.Scale(domain=monsoon_domain)),
    opacity=alt.value(0),
).add_params(
    nearest
)

# Olou highlighted points.
points_olou = alt.Chart(dataset[dataset['dataset'] == 'olou']).transform_filter(
    nearest
).mark_point(
    size=90, strokeWidth=1,
    opacity=1 - opacity_val
).encode(
    x=alt.X('Date:T', scale=alt.Scale(domain=x_domain)),
    y=alt.Y('Counts:Q', scale=alt.Scale(domain=olou_domain)),
    color=alt.Color("dataset", legend=alt.Legend(symbolType='circle'))
).add_params(
    opacity_val,
    select
)

# Olou moving av highlighted points.
points_olou_rolling = alt.Chart(dataset[np.isnan(dataset['Counts']) == False]).transform_window(
    rolling_mean='mean(Counts)',
    frame=[-15, 15],
).transform_filter(
    nearest
).mark_point(
    size=90, strokeWidth=1,
    opacity=opacity_val
).encode(
    x=alt.X('Date:T', scale=alt.Scale(domain=x_domain)),
    y=alt.Y('rolling_mean:Q', scale=alt.Scale(domain=olou_domain)),
    color=alt.Color("dataset", legend=alt.Legend(symbolType='circle'))
).add_params(
    opacity_val,
    select
)

# Monsoon highlighted points.
points_monsoon = alt.Chart(dataset[dataset['dataset'] == 'monsoon']).transform_filter(
    nearest
).mark_point(
    size=90, strokeWidth=1,
    opacity=1 - opacity_val
).encode(
    x=alt.X('Date:T', scale=alt.Scale(domain=x_domain)),
    y=alt.Y('Precip:Q', scale=alt.Scale(domain=monsoon_domain)),
    color=alt.Color("dataset", legend=alt.Legend(symbolType='circle'))
).add_params(
    opacity_val,
    select
)

# Monsoon moving av highlighted points.
points_monsoon_rolling = alt.Chart(dataset[np.isnan(dataset['Precip']) == False]).transform_window(
    rolling_mean='mean(Precip)',
    frame=[-15, 15],
).transform_filter(
    nearest
).mark_point(
    size=90, strokeWidth=1,
    opacity=opacity_val
).encode(
    x=alt.X('Date:T', scale=alt.Scale(domain=x_domain)),
    y=alt.Y('rolling_mean:Q', scale=alt.Scale(domain=monsoon_domain)),
    color=alt.Color("dataset", legend=alt.Legend(symbolType='circle'))
).add_params(
    opacity_val,
    select
)

# Olou label text.
text_olou = alt.Chart(dataset[dataset['dataset'] == 'olou']).mark_text(
    align='left',
    dx=7,
    fontSize=14,
    opacity=1 - opacity_val
).encode(
    x=alt.X('Date:T', scale=alt.Scale(domain=x_domain)),
    y=alt.value(10),
    text=alt.Text('Counts', format='.2f'),
    color=alt.Color("dataset", legend=alt.Legend(symbolType='circle'))
).transform_filter(
    nearest
).add_params(
    opacity_val,
    select
)

# Olou label text (moving av).
text_olou_moving_av = line_chart_olou_rolling.mark_text(
    align='left',
    dx=7,
    fontSize=14,
    opacity=opacity_val
).encode(
    x=alt.X('Date:T', scale=alt.Scale(domain=x_domain)),
    y=alt.value(10),
    text=alt.Text('rolling_mean:Q', format='.2f'),
    color=alt.Color("dataset", legend=alt.Legend(symbolType='circle'))
).transform_filter(
     nearest
).add_params(
    opacity_val,
    select
)

# Monsoon label text.
text_monsoon = alt.Chart(dataset[dataset['dataset'] == 'monsoon']).mark_text(
    align='left',
    dx=7,
    fontSize=14,
    opacity=1 - opacity_val
).encode(
    x=alt.X('Date:T', scale=alt.Scale(domain=x_domain)),
    y=alt.value(10),
    text=alt.Text('Precip', format='.2f'),
    color=alt.Color("dataset", legend=alt.Legend(symbolType='circle'))
).transform_filter(
    nearest
).add_params(
    opacity_val,
    select
)

# Monsoon label text (moving av).
text_monsoon_moving_av = line_chart_monsoon_rolling.mark_text(
    align='left',
    dx=-7,
    fontSize=14,
    opacity=opacity_val
).encode(
    x=alt.X('Date:T', scale=alt.Scale(domain=x_domain)),
    y=alt.value(10),
    text=alt.Text('rolling_mean:Q', format='.2f'),
    color=alt.Color("dataset", legend=alt.Legend(symbolType='circle'))
).transform_filter(
     nearest
).add_params(
    opacity_val,
    select
)

### Legend

In [20]:
# Separate layer for this so that it's not
# affected by the opacity widget.
legend_dataset = alt.Chart(dataset).mark_point(
    opacity=1.0,
    filled=True
).encode(
    y=alt.Y('dataset:N', axis=alt.Axis(orient='right', title='')),
    color=alt.Color('dataset:N', scale=alt.Scale(range=['blue', 'red']))
)

# Flood/drought line
legend_vlines = alt.Chart({'values': [{'y': 'Drought'}, {'z': 'Flood'}]}).mark_rule(
    strokeDash=[2, 2],
    color='black'
).encode(
    strokeDash=alt.StrokeDash('y:N', scale=alt.Scale(domain=['Drought', 'Flood'], range=[[2, 2], []]), title='')
)

### Set Drought/Flood Lines

In [21]:
# Drought years (from paper).
vline_chart_droughts = alt.Chart({'values': [
    {'x': '1965-01-01'},
    {'x': '1966-01-01'},
    {'x': '1968-01-01'},
    {'x': '1972-01-01'},
    {'x': '1974-01-01'},
    {'x': '1982-01-01'},
    {'x': '1986-01-01'},
    {'x': '1987-01-01'},
    {'x': '2002-01-01'},
    {'x': '2004-01-01'},
    {'x': '2009-01-01'}
]}).mark_rule(
    strokeDash=[2, 2],
    color='black'
).encode(
    x=alt.X('x:T'),
)

# Flood years (from paper).
vline_chart_floods = alt.Chart({'values': [
    {'x': '1964-01-01'},
    {'x': '1970-01-01'},
    {'x': '1971-01-01'},
    {'x': '1973-01-01'},
    {'x': '1975-01-01'},
    {'x': '1978-01-01'},
    {'x': '1983-01-01'},
    {'x': '1988-01-01'},
    {'x': '1990-01-01'},
    {'x': '1994-01-01'},
    {'x': '1997-01-01'},
    {'x': '1998-01-01'}
]}).mark_rule(
    color='black'
).encode(
    x=alt.X('x:T'),
)

### Layering Components

In [22]:
# Olou chart.
chart_olou = alt.layer(
    selectors_olou,
    vline_chart_droughts,
    vline_chart_floods,
    line_chart_olou,
    line_chart_olou_rolling,
    points_olou,
    points_olou_rolling,
    rules,
    text_olou,
    text_olou_moving_av
)

# Monsoon chart.
chart_monsoon = alt.layer(
    selectors_monsoon,
    vline_chart_droughts,
    vline_chart_floods,
    line_chart_monsoon,
    line_chart_monsoon_rolling,
    points_monsoon,
    points_monsoon_rolling,
    rules,
    text_monsoon,
    text_monsoon_moving_av
)

### Interactivity

In [23]:
# Init outputs.
output = widgets.Output()

# Update chart sizes.
def update_chart_width_height(width, height):
    updated_chart_olou = chart_olou.properties(width=width, height=height)
    updated_chart_monsoon = chart_monsoon.properties(width=width, height=height)
    updated_combined_chart = alt.vconcat(updated_chart_olou, updated_chart_monsoon)
    updated_combined_chart = alt.hconcat(updated_combined_chart, legend_dataset, legend_vlines)
    with output:
        output.clear_output(wait=True)
        display(updated_combined_chart)

# Create widgets.
width_slider = widgets.IntSlider(value=800, min=100, max=1500, step=1, description='View width:')
height_slider = widgets.IntSlider(value=155, min=10, max=300, step=1, description='View height:')

# Update charts from widgets.
def update_chart(width, height):
    update_chart_width_height(width, height)

# Activate widgets.
widgets.interactive(update_chart, width=width_slider, height=height_slider); # semicolon to supress width/height widget output
layout_widgets = pn.panel(widgets.HBox(children=[width_slider, height_slider]))

### Display

In [24]:
# Render mode.
alt.renderers.enable('default')

# Html.
html_code1 = """
<h1>Cosmic Rays &amp; Monsoons</h1>
<p><big>Is there an obvious relationship between the flux of cosmic rays and rainfall intensity of summer monsoons in India? You can compare both total measurements as well as rolling averages.</big</p>
<hr>
"""

html_code2 = """
<hr>
"""

# Final display.
display(HTML(html_code1), layout_widgets, HTML(html_code2), output)

IPyWidget(HBox)

Output()